#### Notes
- unzip address points gdb before running script
- process of elimination is used to wittle out parcels
- owned unit groupings (OUG) relies parcel common areas
    - if those aren't are accurately attributed, parcels within won't be correctly attributed
- when reusing scripts for later year, qaqc parcel ids used in queries


In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [2]:
# input taz shapefile and parcels
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Salt_Lake_County_Parcels_LIR.gdb\\Parcels_SaltLake_LIR_UTM12'

In [3]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "classes.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "classes.gdb")
    
scratch = os.path.join(outputs, "scratch.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch.gdb")
    
final = os.path.join(outputs, "final.gdb")
if not arcpy.Exists(final):
    arcpy.CreateFileGDB_management(outputs, "final.gdb")

In [4]:
# get address points without base address point
address_pts = '.\\Inputs\\AddressPoints.gdb\\AddressPoints_Salt_Lake'
address_pts_lyr = arcpy.MakeFeatureLayer_management(address_pts,'address_pts_lyr')
query = (''' PtType <> 'BASE ADDRESS' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, scratch, '_00_address_pts_no_base')

In [5]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, scratch, '_01_parcels_for_modeling')

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(parcels_for_modeling, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(parcels_for_modeling, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_lyr') 

# house count field is a string for some reason, so this is the fix
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', """ HOUSE_CNT IS NULL """)
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT", """0""", "PYTHON3")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
arcpy.AddField_management(parcels_for_modeling_layer, 'HOUSE_CNT2', 'LONG')
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT2", """int(!HOUSE_CNT!)""", "PYTHON3")

# add second built year field
arcpy.AddField_management(parcels_for_modeling_layer, 'BUILT_YR2', 'LONG')
arcpy.CalculateField_management(parcels_for_modeling_layer, "BUILT_YR2", """!BUILT_YR!""", "PYTHON3")

<Result 'parcels_for_modeling_lyr'>

In [6]:
# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling_layer, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'SUBTYPE_WFRC', 'TEXT')

# add REMM fields
arcpy.AddField_management(parcels_for_modeling_layer, 'zone_id', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'elevation', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_dua', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_far', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_height', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type1', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type2', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type3', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type4', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type5', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type6', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type7', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type8', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'agriculture', 'SHORT')
arcpy.AddField_management(parcels_for_modeling_layer, 'basebldg', 'SHORT')
arcpy.AddField_management(parcels_for_modeling_layer, 'NoBuild', 'SHORT') # REMM doesn't use this
arcpy.AddField_management(parcels_for_modeling_layer, 'redev_friction', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'building_type_id', 'SHORT')

fields=['basebldg','type1','type2','type3','type4','type5','type6','type7','type8','agriculture','NoBuild']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        row[0] = 0
        row[1] = 'f'
        row[2] = 'f'
        row[3] = 'f'
        row[4] = 'f'
        row[5] = 'f'
        row[6] = 'f'
        row[7] = 'f'
        row[8] = 'f'
        row[9] = 0
        row[10] = 0
        cursor.updateRow(row)

# count parts and rings
arcpy.AddField_management(parcels_for_modeling_layer, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)

# delete parcels with empty geometry
query = (""" PARTS =  0""")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# initial parcels in modeling area:\n {}".format(count_all))  

# initial parcels in modeling area:
 384319


## Categorize parcels within Owned Unit Groupings

In [7]:
###############
# Common Areas
###############

common_areas = r'.\Inputs\salt_lake_common_areas.gdb\salt_lake_common_areas'
common_areas_lyr = arcpy.MakeFeatureLayer_management(common_areas, 'common_areas_lyr') 

# open_space
query = """ TYPE_WFRC IN ('open_space') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_open_space = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02a_ca_open_space')

# vacant
query = """ TYPE_WFRC IN ('vacant') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_vacant = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02b_ca_vacant')

# commercial
query = """ TYPE_WFRC IN ('commercial') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_commercial = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02d_ca_commercial')

# pud
query = """ SUBTYPE_WFRC IN ('pud') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_pud = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02h_ca_pud')

# multi_family
query = """ TYPE_WFRC IN ('multi_family') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_multi_family = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02i_ca_multi_family')

# undevelopable
query = """ TYPE_WFRC IN ('undevelopable') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_undevelopable = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02j_ca_undevelopable')

In [8]:
###############
# PUD
###############

tag = "single_family"
tag2 = "pud"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_pud, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_pud_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_pud, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_pud
join_features = pud_centroids
output_features = os.path.join(scratch, "_03a_pud_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year mode
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(oug_sj, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_pud")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

20112 "single_family" parcels were selected.
364207 parcels remain...


In [9]:
###############
# multi family
###############

tag = "multi_family"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_multi_family, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
mf_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_mf_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_multi_family, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_multi_family
join_features = mf_centroids
output_features = os.path.join(scratch, "_03b_mf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year mode
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_multi_family")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

50524 "multi_family" parcels were selected.
313683 parcels remain...


In [10]:
#####################
# commercial commons
#####################

tag = "commercial"

# parcels that are contained by industrial common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_commercial, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
commercial_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_commercial_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_commercial, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_commercial
join_features = commercial_centroids
output_features = os.path.join(scratch, "_03e_commercial_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year mode
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_commercial_commons")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# update year built with max if mode is 0
with arcpy.da.UpdateCursor(oug_sj2, ['BUILT_YR', 'BUILT_YR2']) as cursor:
    for row in cursor:
        if row[0]  is None or row[0] < 1 or row[0] == "":
            row[0] = row[1]
        
        cursor.updateRow(row)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="4",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

23 "commercial" parcels were selected.
313660 parcels remain...


## Categorize parcels

In [11]:
###########
# churches
###########

query = ("""PROP_TYPE IN ('956') OR TAXEXEMPT_TYPE IN ('ER') """)
tag="churches"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="8",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1442 "churches" parcels were selected.
312218 parcels remain...


In [12]:
###############
# agriculture
###############

query = """ PROP_TYPE IN ('811','812','816','817', '818','830','850') """
tag="agriculture"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1291 "agriculture" parcels were selected.
310927 parcels remain...


In [13]:
###############
# health care
###############

query = (""" PROP_TYPE in ('524', '527','547','560','561') """)
tag = 'healthcare'

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

527 "healthcare" parcels were selected.
310400 parcels remain...


In [14]:
###############
# education
###############

query = (""" PROP_TYPE IN  ('954','952','577') OR TAXEXEMPT_TYPE = 'EE' OR PARCEL_ID IN ('33123520020000', 
            '27251270410000', '21253850060000', '16091030030000') """)
tag="education"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="6",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

711 "education" parcels were selected.
309689 parcels remain...


In [15]:
###############
# government
###############

query = (""" PROP_TYPE IN ('570','953','955') OR PARCEL_ID IN ('28031050020000') """)
tag="government"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="6",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5748 "government" parcels were selected.
303941 parcels remain...


In [16]:
###############
# retail
###############
query=(""" PROP_TYPE IN ('503','507','513','514','515','516','518','523','525','528','529','539','530','536', '537','548','549',
            '551','553','562','564','573','574','575','578','581','582','583','584','585','591','675','749','775','914') 
            OR PARCEL_ID IN ('16073130110000','16073130120000','16073130160000','16073130130000','16073130170000',
            '16073130140000','16073130180000','16073130150000','16073130190000', '26233270030000', '27134260150000', 
            '14241040100000', '22171020030000') """)


tag="retail"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="4",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

7345 "retail" parcels were selected.
296596 parcels remain...


In [17]:
###############
# industrial
###############

query = (""" (PROP_TYPE IN ('200','202','203', '550', '552', '556','695','795','915')) OR (PROP_CLASS = 'I') 
                OR (PARCEL_ID IN ('15173000540000', '16081760150000')) """)
tag="industrial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="3",
                                expression_type="PYTHON3")


# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

6765 "industrial" parcels were selected.
289831 parcels remain...


In [18]:
###############
# office
###############

query= ("""PROP_TYPE IN ('506','566','590', '660', '760','916') OR PARCEL_ID IN ('28271520010000') """)
tag="office"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

4834 "office" parcels were selected.
284997 parcels remain...


In [19]:
################
# single_family
################

query = (''' PROP_TYPE IN ('103','104','111', '118', '511', '651', '888','994','998') OR PARCEL_ID IN ('28304780680000') ''')
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# manual select some strays
query = (''' PARCEL_ID IN ('26253360020000','26253360050000','26253360080000','26253300160000','26253300180000',
'33072760080000','33072780070000','33102280070000','33102310150000') ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# single family - environmental degradation
query = (''' PROP_TYPE IN ('521') AND HOUSE_CNT2 > 0 ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")


# create the feature class for the parcel type
single_family = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

231557 "single_family" parcels were selected.
53440 parcels remain...


In [20]:
########################
# Related Parcel (SF)
########################

# select all related parcels
query= (""" PROP_TYPE IN ('957') AND PARCEL_ADD IS NOT NULL AND PARCEL_ADD <> ' '  """)
tag = "related_parcel"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
related_parcels = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_08_{}'.format(tag))

# merge related parcels into dc and dissolve if addresses match
merged = arcpy.Merge_management([single_family, related_parcels], os.path.join(scratch, '_08_merged_sf_related'))
merged_layer = arcpy.MakeFeatureLayer_management(merged, 'merged_layer')

# get parcels that have an address
query= (""" PARCEL_ADD IS NOT NULL AND PARCEL_ADD <> ' '  """)
arcpy.SelectLayerByAttribute_management(merged_layer, 'NEW_SELECTION', query)

# dissolve parcels that have addresses
dissolved_parcels = arcpy.Dissolve_management(merged_layer, os.path.join(scratch, '_08_merged_dissolve'), 
                                              ["PARCEL_ADD"], [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], 
                                              ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'],['FLOORS_CNT', 'MAX'],
                                              ['BUILT_YR', 'MAX'], ['HOUSE_CNT2', 'SUM'],
                                              ['PROP_TYPE', 'MIN'],['TYPE_WFRC', 'FIRST'],['PARCEL_ID', 'FIRST']])

# invert selection and dissolve using OID
arcpy.SelectLayerByAttribute_management(merged_layer, 'SWITCH_SELECTION')
dissolved_parcels2 = arcpy.Dissolve_management(merged_layer, os.path.join(scratch, '_08_merged_dissolve2'), 
                                              ["PARCEL_ADD"], [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], 
                                              ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'],['FLOORS_CNT', 'MAX'],
                                              ['BUILT_YR', 'MAX'], ['HOUSE_CNT2', 'SUM'],
                                              ['PROP_TYPE', 'MIN'],['TYPE_WFRC', 'FIRST'],['PARCEL_ID', 'FIRST']])

sf_rp = os.path.join(scratch, '_02_single_family_with_rp')
merged2 = arcpy.Merge_management([dissolved_parcels, dissolved_parcels2], sf_rp)
merged2_layer = arcpy.MakeFeatureLayer_management(merged2, 'merged2_layer') 

# remove related parcels that did not join to others
query= (""" FIRST_TYPE_WFRC IN ('related_parcel') """)
arcpy.SelectLayerByAttribute_management(merged2_layer, 'NEW_SELECTION', query)
merged2_layer = arcpy.DeleteFeatures_management(merged2_layer)


# calculate basebldg field
arcpy.CalculateField_management(sf_rp, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(sf_rp, field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# remove parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged2_layer, selection_type='NEW_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

4476 "related_parcel" parcels were selected.
48964 parcels remain...


In [21]:
##############################
# Related Parcel (SF) cont'd
##############################

sf_rp_sdf = pd.DataFrame.spatial.from_featureclass(sf_rp)

sf_rp_sdf.columns = ['OBJECTID', 'PARCEL_ADD', 'TOTAL_MKT_VALUE', 'LAND_MKT_VALUE',
       'PARCEL_ACRES', 'BLDG_SQFT','FLOORS_CNT', 'BUILT_YR',
       'HOUSE_CNT2', 'PROP_TYPE', 'TYPE_WFRC', 'PARCEL_ID',
       'basebldg', 'building_type_id', 'SHAPE']

sf_rp_sdf = sf_rp_sdf[['PARCEL_ADD', 'TOTAL_MKT_VALUE', 'LAND_MKT_VALUE',
       'PARCEL_ACRES', 'BLDG_SQFT','FLOORS_CNT', 'BUILT_YR',
       'HOUSE_CNT2', 'PROP_TYPE', 'TYPE_WFRC', 'PARCEL_ID',
       'basebldg', 'building_type_id', 'SHAPE']].copy()

sf_rp_sdf['TYPE_WFRC'] = 'single_family'
sf_rp_sdf['SUBTYPE_WFRC'] = 'single_family'

sf_rp_sdf.spatial.to_featureclass(location=os.path.join(gdb, '_02_single_family_with_rp'),sanitize_columns=False)

'E:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-SaltLake\\Outputs\\classes.gdb\\_02_single_family_with_rp'

In [22]:
########################
# single_family w/ ADU
########################

query = (''' PROP_TYPE IN ('106') ''')
tag="single_family"
tag2="single_family_adu"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag2))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag2)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag2)), field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag2, count_remaining))

1439 "single_family_adu" parcels were selected.
47525 parcels remain...


In [23]:
################
# duplex
################

query = (''' PROP_TYPE IN ('112', '512') ''')
tag="duplex"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5397 "duplex" parcels were selected.
42128 parcels remain...


In [24]:
################
# Residential Multi
################

# multiple buildings/residences on one parcel

tag="multi_family"
tag2="apartments"
tag3="residential_multi"

query = (''' PROP_TYPE IN ('997') ''')

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)



# create the feature class for the parcel type
features = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_02_{}'.format(tag3))


#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = features
join_features = address_pts_no_base
output_features = os.path.join(gdb, '_02_{}'.format(tag3))

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                                     match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")


# calculate the type field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='Note', expression="'{}'".format(tag3),
                                expression_type="PYTHON3")

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag3)), field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag3, count_remaining))

532 "residential_multi" parcels were selected.
41596 parcels remain...


In [25]:
##########################
# Residential Salvage
##########################

query = (''' PROP_TYPE IN ('501', '993') ''')
tag="residential_salvage"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# export ids to csv to adjust friction factor
# In disrepair, will likely need to be demolished
# Set the friction factor for these to be lower

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

79 "residential_salvage" parcels were selected.
41517 parcels remain...


In [26]:
###############
# utilities
###############

# select parcels using query
query = (''' PROP_TYPE IN ('950', '958') ''')
tag="utilities"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

13 "utilities" parcels were selected.
41504 parcels remain...


In [27]:
#################
# group quarters
#################

query= (""" PROP_TYPE IN ('105','540','576') """)
tag = "group_quarters"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

144 "group_quarters" parcels were selected.
41360 parcels remain...


In [28]:
######################
# Apartments
######################

tag = "multi_family"
tag2 = "apartments"

query= (""" PROP_TYPE IN ('110','113','114', '115', '120', '150', '199') OR
            PARCEL_ID IN ('16294330010000', '16292380050000') """)


# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(parcels_for_modeling_layer, field='Note', expression="!PROP_TYPE!",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
apt_commons = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_03_{}'.format(tag))


##############

query = """ PROP_TYPE IN ('504','713', '913') or PARCEL_ID IN ('16294330010000') """

# filter for common areas
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")


common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_03a_apt_commons')

# fill in holes via eliminate polygon part
apt_commons2 = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(scratch, '_03b_apt_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')


#=========================
# get merge apt commons
#=========================

merged_apt_commons = arcpy.Merge_management([apt_commons, apt_commons2], 
                                              os.path.join(scratch, '_03c_merged_apt_commons'))
# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(merged_apt_commons),"PARCEL_ACRES" "AREA",'', 'ACRES')   
arcpy.CalculateField_management(merged_apt_commons, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON", )

#==================================================
# summarize units attributes within common areas
#==================================================

# get parcels that are contained by apt common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
apt_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03d_apt_centroids'), "INSIDE")




# use spatial join to summarize market value & acreage
target_features = merged_apt_commons
join_features = apt_centroids
output_features = os.path.join(scratch, "_03e_apt_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year mode
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# built year max
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
                                expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_apartment")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")



#=========================
# final summary
#=========================

# update year built with max if mode is 0
codeblock = """def replaceYear(mode_year,max_year):
                 if mode_year is not None:
                   if mode_year < 1:  
                     return max_year
                 else:
                   return max_year
                 """
arcpy.CalculateField_management(oug_sj2, 'BUILT_YR', "replaceYear(!BUILT_YR!, !BUILT_YR2!)".format(tag), 
                                "PYTHON3", codeblock)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, "_02_apartment"), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, "_02_apartment"), field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged_apt_commons, selection_type='ADD_TO_SELECTION')


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# select all features within merged dataset and delete
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag2, count_remaining))

5545 "apartments" parcels were selected.
35815 parcels remain...


In [29]:
# ################################
# # OUG - Condos
# ################################

# tag = "condo"

# condo_groupings_reviewed = r'.\Inputs\ougs_reviewed.gdb\condos_reviewed2'


# # recalc acreage
# #arcpy.AddField_management(condo_groupings_reviewed, 'PARCEL_ACRES', 'FLOAT')
# # arcpy.CalculateGeometryAttributes_management(condo_groupings_reviewed, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
# arcpy.CalculateField_management(condo_groupings_reviewed, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

# #==================================================
# # summarize units attributes within common areas
# #==================================================

# # parcels that are contained by condo common areas
# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
#                                        select_features=condo_groupings_reviewed, selection_type='NEW_SELECTION')

# # calculate the type field
# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
#                                 expression_type="PYTHON3")

# # convert condo parcels that are contained by common areas into centroids
# condo_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
#                                                   os.path.join(scratch, '_04j_condos_centroids'), "INSIDE")



# # use spatial join to summarize market value & acreage
# target_features = condo_groupings_reviewed
# join_features = condo_centroids
# output_features = os.path.join(scratch, "_04k_condo_oug_sj")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# # total market value
# fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # # parcel acres
# # fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# # fieldmap = fieldmappings.getFieldMap(fieldindex)
# # fieldmap.mergeRule = 'Sum'
# # fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # year
# fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Mode'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)


# # run the spatial join, use 'Join_Count' for number of units
# oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
#                            fieldmappings, "CONTAINS")

# # rename join_count
# arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
#                                 expression_type="PYTHON3")
# arcpy.DeleteField_management(oug_sj, "Join_Count")

# #################################
# # get count from address points
# #################################

# # summarize address points address_point_count "ap_count"
# target_features = oug_sj 
# join_features = address_pts_no_base
# output_features = os.path.join(gdb, "_02_condo")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
#                            fieldmappings, match_option="INTERSECTS")

# arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
# arcpy.DeleteField_management(oug_sj2, "Join_Count")

# #################################
# # WRAP-UP
# #################################

# # delete features from working parcels
# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
#                                        select_features=oug_sj2, selection_type='NEW_SELECTION')
# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
#                                        select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

# count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
# parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# # count of remaining parcels
# count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# # message
# print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [30]:
#####################
# low income apartments
#####################

tag="multi_family"
tag2 = "apartments"
tag3 = "low_income_housing"

query= (""" PROP_TYPE IN ('142') """)

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='Note', expression="'{}'".format(tag3),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features = arcpy.EliminatePolygonPart_management(parcels_for_modeling_layer, os.path.join(scratch, '_06a_{}'.format(tag)),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')
layer = arcpy.MakeFeatureLayer_management(features, 'layer') 


# roundabout way to eliminate individual units inside low income common areas
# query= (""" TAXEXEMPT_TYPE = 'MC' """)
# arcpy.SelectLayerByAttribute_management(layer, 'NEW_SELECTION', query)
# common_areas = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_06b_{}_commons'.format(tag))


arcpy.SelectLayerByLocation_management(in_layer=layer,overlap_type="WITHIN",
                                       select_features=features,
                                       selection_type='NEW_SELECTION')
layer = arcpy.DeleteFeatures_management(layer)

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(features, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(features, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = features
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_LowIncomeApts")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")


# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, "_02_LowIncomeApts"), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, "_02_LowIncomeApts"), field='building_type_id', expression="2",
                                expression_type="PYTHON3")


# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag3, count_remaining))

225 "low_income_housing" parcels were selected.
35590 parcels remain...


In [31]:
#################
# mobile home parks
#################

tag = "multi_family"
tag2 = "mobile_home_park"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')

query= (""" PROP_CLASS IN ('118- MOBILE HOME') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mhp = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_07a_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(mhp, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(mhp, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mhp
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_mobile_home_park")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

51 "multi_family" parcels were selected.
35539 parcels remain...


In [32]:
#################
# commercial
#################

# query= (""" PROP_TYPE IN ('500', '510', '520', '546','558','592', '593', '594', '595', '596') OR TAXEXEMPT_TYPE IN ('CC') 
#             OR PROP_CLASS = 'C' """)
query= (""" PROP_TYPE IN ('500', '510', '520', '546','558','592', '593', '594', '595', '596') OR TAXEXEMPT_TYPE IN ('CC')""")
tag="commercial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")
# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="4",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1055 "commercial" parcels were selected.
34484 parcels remain...


In [33]:
#################
# roads
#################

query=""" (BLDG_SQFT = 0 And FLOORS_CNT = 0 And TOTAL_MKT_VALUE = 0 And PARCEL_ADD IS NULL) OR 
            PARCEL_ID IN ('16202760760000') """
tag="road"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

15659 "road" parcels were selected.
18825 parcels remain...


In [34]:
#################
# Parking Structures
#################

query= (""" PROP_TYPE IN ('567', '904', '920') OR PARCEL_ID IN ('27201800030000') """)
tag = "parking_structure"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# could possibly add redev friction to them to make them slightly harder to develop

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

220 "parking_structure" parcels were selected.
18605 parcels remain...


In [35]:
#################
# Undevelopable
#################

tag = "undevelopable"

query= (""" PROP_TYPE IN ('711', '958', '999') OR TAXEXEMPT_TYPE IN ('IR', 'GB') OR PROP_TYPE = ' ' """)

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# contaminated; require environmental remediation
query = (''' PROP_TYPE IN ('521') AND HOUSE_CNT2 < 1 ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# ===================================================
# use  common area to select undevelopable parcels
#====================================================
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=ca_undevelopable,
                                       selection_type='ADD_TO_SELECTION')

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="WITHIN",
                                       select_features=ca_undevelopable,
                                       selection_type='ADD_TO_SELECTION')



# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1933 "undevelopable" parcels were selected.
16672 parcels remain...


In [36]:
####################
# Drop small parcels
####################

query = """ Shape_Area < 375 """

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
print('{} small parcels were deleted.\n{} parcels remain...'.format(count_type, count_remaining))

5235 small parcels were deleted.
11437 parcels remain...


In [37]:
###############
# open space
###############

# get parcels on parks
parks = ".\\Inputs\\ParksLocal.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=parks,
                                       selection_type='NEW_SELECTION')

# get parcels on public lands that are not military
land_ownership_lyr = arcpy.MakeFeatureLayer_management('./Inputs/UT_SITLA_Ownership_LandOwnership_WM.shp','land_ownership_lyr') 

query= (""" OWNER <> 'Private' And DESIG <> 'Military' """)
arcpy.SelectLayerByAttribute_management(land_ownership_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=land_ownership_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get cemetery parcels using overlay
cemeteries_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\Cemeteries.shp','cemeteries_lyr') 
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=cemeteries_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get parcels that are golf courses using overlay
golf_courses = ".\\Inputs\\GolfCourses.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=golf_courses,
                                       selection_type='ADD_TO_SELECTION')

#=====================================================
# get parcels that overlap with reviewed common spaces
#======================================================

# open space
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=ca_open_space,
                                       selection_type='ADD_TO_SELECTION')

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="WITHIN",
                                       select_features=ca_open_space,
                                       selection_type='ADD_TO_SELECTION')

#============================
# query parcel attributes
#============================

query= (""" PROP_TYPE IN ('951','960','961') OR TAXEXEMPT_TYPE = 'CM' """)
tag = "open_space"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

3954 "open_space" parcels were selected.
7483 parcels remain...


In [38]:
#################
# Vacant land
#################

tag = "vacant"
# query= (""" PROP_TYPE IN ('119','901','902','903', '905', '906', '908', '911', '917', '922') or (PROP_TYPE IS NULL) """)

# # select the features
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# #========================================================
# # get parcels that overlap with reviewed common spaces
# #========================================================
# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
#                                        select_features=ca_vacant,
#                                        selection_type='ADD_TO_SELECTION')

# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="WITHIN",
#                                        select_features=ca_vacant,
#                                        selection_type='ADD_TO_SELECTION')

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

7483 "vacant" parcels were selected.
0 parcels remain...


In [39]:
# ############################
# # Community Land Trust (SF)
# ############################

# # select all 651 parcels
# query= (""" PROP_TYPE IN ('651') AND PARCEL_ADD IS NOT NULL AND PARCEL_ADD <> ' '  """)
# tag = "comm_land_trust"

# # select the features
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# # calculate the type field
# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
#                                 expression_type="PYTHON3")

# # create the feature class for the parcel type
# clt = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_08_{}'.format(tag))

# # dissolve parcels that have addresses
# dissolved_parcels = arcpy.Dissolve_management(clt, os.path.join(gdb, '_02_{}'.format(tag)), 
#                                               ["PARCEL_ADD"], [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], 
#                                               ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'], ['BUILT_YR', 'MAX'],
#                                               ['BUILT_YR', 'SUM'], ['HOUSE_CNT2', 'SUM'],['PROP_TYPE', 'MIN'], 
#                                               ['TYPE_WFRC', 'FIRST']])

# dp_layer = arcpy.MakeFeatureLayer_management(dissolved_parcels, 'dp_layer') 


# # remove parcels
# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
#                                        select_features=dp_layer, selection_type='NEW_SELECTION')
# count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
# parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
# count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

In [40]:
# # export remaining parcels
# arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_09_unclassed_parcels')

In [41]:
##################################
# Merge the parcels back together
##################################

arcpy.env.workspace = gdb
featureclasses = arcpy.ListFeatureClasses()
merged_parcels = arcpy.Merge_management(featureclasses, os.path.join(scratch, 'merged_parcels'),
                                        add_source='ADD_SOURCE_INFO')
merged_parcels_lyr = arcpy.MakeFeatureLayer_management(merged_parcels, 'merged_parcels_lyr') 

## Use the Generalized Future Land Use layer to set the parcel zoning

In [42]:
# set land use type on parcels using glfu
gflu = '.\Inputs\FutureLandUse2020.gdb\FutureLandUse2020'
gflu_lyr = arcpy.MakeFeatureLayer_management(gflu, 'gflu_lyr') 


##################
# Type 1 (SF)
##################

# query for land use type
query = """ GenLUType IN ('Any Development','Any Residential', 'Mixed Use SF', 'Residential SF', 'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type1', expression="'t'".format(tag),
                                expression_type="PYTHON3")


##################
# Type 2 (MF)
##################

# query for land use type
query = """ GenLUType IN ('Any Residential', 'Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 
                          'Residential MF/Office', 'Residential MF', 'Mixed Use MF', 'Mixed Use SF', 'Residential/Office', 
                          'Residential/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type2', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#######################
# Type 3 (Industrial)
#######################

# query for land use type
query = """ GenLUType IN ('Industrial', 'Any Development', 'Industrial/Mixed Use MF') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type3', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#######################
# Type 4 (Retail)
#######################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
                          'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type4', expression="'t'".format(tag),
                                expression_type="PYTHON3")


#######################
# Type 5 (Office)
#######################

# query for land use type
query = """ GenLUType IN ('Retail/Office', 'Office', 'Any Commercial', 'Residential/Office', 'Residential MF/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type5', expression="'t'".format(tag),
                                expression_type="PYTHON3")

####################################
# Type 6 (Government and Education)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Government/Education') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type6', expression="'t'".format(tag),
                                expression_type="PYTHON3")


####################################
# Type 7 (Mixed Use)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 'Mixed Use MF', 'Mixed Use SF', 
                          'Residential MF/Office', 'Residential SF/Retail', 'Residential/Office', 'Residential/Retail', 
                          'Retail/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type7', expression="'t'".format(tag),
                                expression_type="PYTHON3")

####################################
# Type 8 (Other)
###################################

# # query for land use type
# query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
#                           'Residential SF/Retail') """
# arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# # select the parcels
# arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
#                                        select_features=gflu_lyr,
#                                        selection_type='NEW_SELECTION')

# arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'t'".format(tag),
#                                 expression_type="PYTHON3")

#################################
# Undevelopable
#################################

# query for land use type
query = """ GenLUType IN ('NoBuild') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr, overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

query = """ NoBuild = 1 """
arcpy.SelectLayerByAttribute_management(merged_parcels_lyr, 'NEW_SELECTION', query)


# set all types to false
arcpy.CalculateField_management(merged_parcels_lyr, field='type1', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type2', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type3', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type4', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type5', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type6', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type7', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'f'".format(tag),
                                expression_type="PYTHON3")

# clear selected parcels
arcpy.SelectLayerByAttribute_management(merged_parcels_lyr, 'CLEAR_SELECTION', query)
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'CLEAR_SELECTION', query)

id,value
0,a Layer object
1,-1


In [43]:
#######################################################
# run spatial join to get maxdua and maxfar from gflu
#######################################################

target_features = merged_parcels_lyr
join_features = gflu_lyr
output_features = os.path.join(scratch, "_08_parcel_glfu_join")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="HAVE_THEIR_CENTER_IN")


parcel_glfu_join = pd.DataFrame.spatial.from_featureclass(oug_sj2)[['OBJECTID', 'MaxDUA']].copy()

## Eject buildings from parcel data

In [44]:
parcels_buildings_sdf = pd.DataFrame.spatial.from_featureclass(merged_parcels)

#######################
# add max dua
#######################

parcels_buildings_sdf = parcels_buildings_sdf.merge(parcel_glfu_join, left_on='OBJECTID', right_on='OBJECTID', how='inner')
parcels_buildings_sdf['max_dua'] = parcels_buildings_sdf['MaxDUA']
del parcels_buildings_sdf['MaxDUA']

#######################
# Fix the unit counts
#######################

# convert unit count columns to int
parcels_buildings_sdf.loc[(parcels_buildings_sdf['HOUSE_CNT'].isnull() == True ), 'HOUSE_CNT'] = 0
parcels_buildings_sdf.loc[(parcels_buildings_sdf['ap_count'].isnull() == True ), 'ap_count'] = 0
parcels_buildings_sdf['ap_count'] = parcels_buildings_sdf['ap_count'].astype(int)


# create new count field and calculate
parcels_buildings_sdf['UNIT_COUNT'] = parcels_buildings_sdf['ap_count']

# fix single family (non-pud)
parcels_buildings_sdf.loc[(parcels_buildings_sdf['UNIT_COUNT'] == 0) & 
                 (parcels_buildings_sdf['TYPE_WFRC'] == 'single_family'), 
                 'UNIT_COUNT'] = 1

# fix duplex
parcels_buildings_sdf.loc[(parcels_buildings_sdf['SUBTYPE_WFRC'] == 'duplex'), 'UNIT_COUNT'] = 2


###############################
# Subset to necessary columns
###############################

parcels_buildings_sdf = parcels_buildings_sdf[['COUNTY_NAME','COUNTY_ID','PARCEL_ID','TOTAL_MKT_VALUE',
                                               'LAND_MKT_VALUE','PARCEL_ACRES','UNIT_COUNT','BLDG_SQFT',
                                               'FLOORS_CNT','BUILT_YR','EFFBUILT_YR','max_dua','max_far','max_height', 
                                               'type1','type2','type3','type4','type5','type6','type7','type8', 
                                               'agriculture','basebldg','NoBuild','redev_friction','building_type_id','x','y',
                                               'SHAPE']].copy()



#############################################
# won't need these till all counties are done
#############################################

# create new REMM parcel IDs
parcels_buildings_sdf['parcel_id_remm'] = np.arange(len(parcels_buildings_sdf))

# copy table and only keep rows if basebldg = 1
buildings_df = parcels_buildings_sdf[parcels_buildings_sdf['basebldg'] == 1].copy()
buildings_df = buildings_df[['parcel_id_remm','BLDG_SQFT','TOTAL_MKT_VALUE', 'LAND_MKT_VALUE','UNIT_COUNT',
                             'building_type_id', 'FLOORS_CNT','BUILT_YR']].copy()

# calc building value?
buildings_df['B_VALUE'] = buildings_df['TOTAL_MKT_VALUE'] - buildings_df['LAND_MKT_VALUE']

# generate building id
buildings_df['building_id'] = np.arange(len(buildings_df))

# add new attributes
buildings_df['non_residential_sqft'] = 0
buildings_df['note'] = 'base'
buildings_df['residential_units'] = 0
buildings_df['unit_price_non_residential'] = 0
buildings_df['job_spaces'] = 0

buildings_df.loc[buildings_df['building_type_id'].isin([1,2])== False, 'non_residential_sqft'] = buildings_df['BLDG_SQFT']
buildings_df.loc[buildings_df['building_type_id'].isin([1,2])== True, 'residential_units'] = buildings_df['UNIT_COUNT']

# rename building columns
buildings_df = buildings_df.rename(columns={"BLDG_SQFT": "bldg_sqft", "FLOORS_CNT": "stories",'BUILT_YR':'year_built'})

In [45]:
# export
parcels_buildings_sdf.spatial.to_featureclass(location= os.path.join(final, 'salt_lake_parcels'),sanitize_columns=False)
parcels_buildings_sdf = parcels_buildings_sdf.drop(columns=['SHAPE', 'building_type_id'])
parcels_buildings_sdf.to_csv(os.path.join(outputs, 'salt_lake_parcels.csv'), index=False)
buildings_df.to_csv(os.path.join(outputs, 'salt_lake_buildings.csv'), index=False)